In [8]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [9]:
#retrieving the url for FOMC Statement
html_page= requests.get('https://www.federalreserve.gov/newsevents/pressreleases/monetary20190320a.htm') 
soup= BeautifulSoup(html_page.content, 'html.parser')

In [10]:
#preview the soup... MMM soup
print(soup.prettify()[:1000])

<!DOCTYPE doctype html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0 maximum-scale=1.6, user-scalable=1" name="viewport"/>
  <meta content=" Information received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity" name="description">
   <meta content="Federal Reserve issues FOMC statement" property="og:title">
    <meta content="Board of Governors of the Federal Reserve System" property="og:site_name">
     <meta content="article" property="og:type"/>
     <meta content=" Information received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity" property="og:description"/>
     <meta content="" property="og:image"/>
     <meta content="summary" name="twitter:card"/>


In [11]:
#search for the text in the article, isolate and save text
article = soup.find("div",attrs={"class":"col-xs-12 col-sm-8 col-md-8"}).getText()

In [12]:
article

"\nInformation received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter. Payroll employment was little changed in February, but job gains have been solid, on average, in recent months, and the unemployment rate has remained low. Recent indicators point to slower growth of household spending and business fixed investment in the first quarter. On a 12-month basis, overall inflation has declined, largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent. On balance, market-based measures of inflation compensation have remained low in recent months, and survey-based measures of longer-term inflation expectations are little changed.\nConsistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. In support of these goals, the Committee decided to ma

In [13]:
import requests
r = requests.get('https://www.federalreserve.gov/json/ne-press.json')

In [14]:
c = r.content

In [15]:
from bs4 import BeautifulSoup

In [16]:
json_file = BeautifulSoup(c)

In [17]:
import json

In [18]:
json.loads(json_file.text[0:3579] + ']')

[{'d': '3/25/2019 2:00:00 PM',
  't': 'Federal and state financial regulatory agencies issue interagency statement on supervisory practices regarding financial institutions affected by flooding in the Midwest',
  'pt': 'Banking and Consumer Regulatory Policy',
  'l': '/newsevents/pressreleases/bcreg20190325a.htm'},
 {'d': '3/22/2019 5:00:00 PM',
  't': 'Federal Reserve Board announces approval of proposals by First Interstate Bank',
  'pt': 'Orders on Banking Applications',
  'l': '/newsevents/pressreleases/orders20190322a.htm'},
 {'d': '3/22/2019 12:00:00 PM',
  't': 'Federal Reserve System publishes annual financial statements',
  'pt': 'Other Announcements',
  'l': '/newsevents/pressreleases/other20190322a.htm'},
 {'d': '3/21/2019 12:00:00 PM',
  't': 'Federal Reserve Board publishes report containing summary information on debit card transactions in 2017',
  'pt': 'Other Announcements',
  'l': '/newsevents/pressreleases/other20190321a.htm'},
 {'d': '3/21/2019 11:00:00 AM',
  't': '

In [19]:
our_list = json_file.text.split("Federal Reserve issues FOMC statement")

In [20]:
links = []
for element in our_list[1:]:
    links.append('https://www.federalreserve.gov/newsevents' + element.split('newsevents')[1][0:36])
links

['https://www.federalreserve.gov/newsevents/pressreleases/monetary20190320a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20190130a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20181219a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20181108a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180926a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180801a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180613a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180502a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180321a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180131a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171213a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171101a.htm',
 'https://www.fe

In [21]:
len('/pressreleases/bcreg20190325a.htm')

33

In [25]:
documents=[]
for link in links:
    html_page= requests.get(link) 
    soup= BeautifulSoup(html_page.content, 'html.parser')
    article = soup.find("div",attrs={"class":"col-xs-12 col-sm-8 col-md-8"}).getText()
    documents.append(article)

In [34]:
documents[0]

"\nInformation received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter. Payroll employment was little changed in February, but job gains have been solid, on average, in recent months, and the unemployment rate has remained low. Recent indicators point to slower growth of household spending and business fixed investment in the first quarter. On a 12-month basis, overall inflation has declined, largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent. On balance, market-based measures of inflation compensation have remained low in recent months, and survey-based measures of longer-term inflation expectations are little changed.\nConsistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. In support of these goals, the Committee decided to ma